### Описание задания.

#### Часть 1: Составление словарей для классификации по тональности.

При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.

Данные для задания: датасет отзывов на банки с сайта banki.ru. Данные содержат непосредственно тексты отзывов, некоторую дополнительную информацию, а также оценку по шкале от 1 до 5. Тексты хранятся в json-ах в массиве responses.

- Разбить всю коллекцию отзывов на предложения. Лемматизировать все слова.
- Обучить по коллекции предложений word2vec.
- Привести несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    - тест на определение ближайших слов
    - тест на аналогии (мужчина – король : женщина – королева)
    - тест на определение лишнего слова.

- Построить несколько визуализаций:
    - TSNE для топ-100 (или топ-500) слов и найти осмысленные кластеры слов
    - задать координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.


#### Часть 2:  Распространение метки.

- Определить 5–8 позитивных слов (например, быстрый, удобный) и 5–8 негативных слов (например, очередь, медленно). Эти слова будут основной будущего оценочного словаря
- Пусть позитивному классу соответствует метка 1, негативному — -1
- Пометить выбранные слова в лексическом графе соответствующими метками
- Запустить любой известный вам метод распространения метки (Label Propogation) в лексическом графе
- На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 — это и есть искомые оценочные слова

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:
- graphlab.label_propagation (graphlab доступен бесплатно по образовательной лицензии)
- sklearn.semi_supervised.LabelPropagation
- sklearn.semi_supervised.LabelSpreading

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import bz2
import igraph as ig
import gdown
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pymorphy2
import re
import regex

from collections import Counter
from contextlib import redirect_stdout
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from string import punctuation
from tqdm import tqdm

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

from gensim import similarities
from gensim.corpora import Dictionary
from gensim.models import KeyedVectors, lsimodel, TfidfModel, word2vec

from sklearn.manifold import TSNE
from sklearn.metrics  import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import LabelPropagation, LabelSpreading

#### Необходимые функции

In [3]:
mystopwords = stopwords.words('russian')

ru_words = re.compile("[А-Яа-я]+")

def words_only(text):
    return " ".join(ru_words.findall(text))

def lemmatize(text):
    try:
        return  " ".join([morph.parse(word)[0].normal_form for word in text.lower().split()])
    except:
        return " "

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""
 
def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))

#### Скачивание данных

In [4]:
url = 'https://drive.google.com/uc?id=1OelGGXPXBinXvZnDb1Bmmxe4rtdBmqdk'

In [5]:
gdown.download(url, quiet=True)

'banki_responses.json.bz2'

In [6]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [00:53, 3786.88it/s]


In [7]:
len(responses)

153499

In [8]:
df = pd.DataFrame(responses)
df.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


### Часть 1: Составление словарей для классификации по тональности.
#### Разбить всю коллекцию отзывов на предложения. Лемматизировать все слова.

In [9]:
sentences = []
for i in range(len(df)):
    sentences.extend(sent_tokenize(df.text.iloc[i]))

In [10]:
sentences[:5]

['Добрый день!',
 'Я не являюсь клиентом банка и поручителем по кредитному договору, а также не являюсь каким-либо другим лицом, письменно  оформившим отношения с банком по поводу урегулирования чьей-либо  задолженности.',
 'Начиная с марта 2015 года начали приходить бесконечные письма из ООО "Примо коллект"на мой адрес: город Москва, Уваровский переулок, дом 10, квартира 111, с угрозами о возбуждении уголовного дела в отношении гражданина Филиппова Эдуарда Владимировича, который уклоняется от уплаты взятых им кредитов: договор № 81014 от 20.10.2013 года и договор № 2464946 от 09.10.2014 года.',
 'Со всей ответственностью\xa0 хочу Вас заверить, что вышеуказанный гражданин, которого Вы разыскиваете, мне не знаком и никогда в моем адресе не был зарегистрирован.',
 'Каким образом Вы не удостоверившись в подлинности его документов оформили на его имя кредитный договор, мне по меньшей мере не понятно,\xa0 и почему по Вашей милости я должна переживать и бояться за себе и свое имущество.']

In [11]:
morph = pymorphy2.MorphAnalyzer()

In [12]:
lemmas = []
for sentence in tqdm(sentences):
    lemmas.append(preprocess(sentence))

100%|█████████████████████████████████████████████████████████████████████| 2664065/2664065 [3:29:52<00:00, 211.56it/s]


In [13]:
lemmas[:5]

['добрый день',
 'являться клиент банк поручитель кредитный договор также являться либо лицо письменно оформить отношение банк повод урегулирование чей либо задолженность',
 'начинать март год начать приходить бесконечный письмо ооо прить коллект адрес город москва уваровский переулок дом квартира угроза возбуждение уголовный дело отношение гражданин филиппов эдуард владимирович который уклоняться уплата взять кредит договор год договор год',
 'весь ответственность хотеть заверить вышеуказанный гражданин который разыскивать знак адрес зарегистрировать',
 'образ удостовериться подлинность документ оформить имя кредитный договор малый мера понятно почему ваш милость должный переживать бояться свой имущество']

#### Обучение по коллекции предложений word2vec.

In [14]:
words = [sentence.split() for sentence in lemmas]

In [15]:
model_response = word2vec.Word2Vec(words, workers=4, vector_size=200, min_count=3, window=5, epochs=15)

In [16]:
model_response.save("model_w2v.model")

In [17]:
#model_response = word2vec.Word2Vec.load("model_w2v.model")

#### Привести несколько удачных и неудачных примеров решения стандартных текстов для word2vec.

In [18]:
model_response.wv.most_similar('доход')

[('заработок', 0.7347224354743958),
 ('ндфл', 0.6583331227302551),
 ('оклад', 0.5795394778251648),
 ('зп', 0.557748556137085),
 ('зарплата', 0.5568051934242249),
 ('прибыль', 0.5516744256019592),
 ('иждивенец', 0.5078229308128357),
 ('среднемесячный', 0.4975089430809021),
 ('доходность', 0.4896053373813629),
 ('оборот', 0.48847496509552)]

In [19]:
model_response.wv.most_similar(positive=['сотрудник', 'человек'], negative=['работник'])

[('парень', 0.5964722037315369),
 ('женщина', 0.573771595954895),
 ('особа', 0.5704456567764282),
 ('жь', 0.539835512638092),
 ('мужчина', 0.5167182683944702),
 ('дама', 0.5116308331489563),
 ('мужик', 0.507178544998169),
 ('девушка', 0.4625983238220215),
 ('мадам', 0.46033748984336853),
 ('паренёк', 0.4601982533931732)]

In [20]:
model_response.wv.doesnt_match('вклад прибыль доход рост кредит'.split())

'кредит'

In [21]:
print(model_response.wv.similarity('процент', 'инфляция'))
print(model_response.wv.similarity('заработок', 'инфляция'))

0.36673552
0.47664872


**Вывод:** тесты показали, что модель хорошо обучилась, хотя мы видим, странное слово 'жь' во втором тесте.

### Построение визуализаций:
- TSNE для топ-500 слов и найти осмысленные кластеры слов.

In [22]:
top_words = []
fd = FreqDist()
for word in words:
    fd.update(word)
for idx in fd.most_common(500):
    top_words.append(idx[0])
print(top_words)

['банк', 'карта', 'это', 'деньга', 'день', 'всё', 'кредит', 'который', 'отделение', 'клиент', 'сотрудник', 'мочь', 'счёт', 'свой', 'сказать', 'сумма', 'заявление', 'год', 'вопрос', 'время', 'ещё', 'получить', 'кредитный', 'весь', 'ответ', 'вклад', 'офис', 'телефон', 'платёж', 'договор', 'ваш', 'дать', 'номер', 'звонить', 'банкомат', 'сбербанк', 'очень', 'позвонить', 'г', 'месяц', 'написать', 'проблема', 'работать', 'средство', 'рубль', 'работа', 'документ', 'ситуация', 'звонок', 'прийти', 'решить', 'говорить', 'человек', 'хотеть', 'операция', 'почему', 'должный', 'информация', 'сделать', 'смс', 'срок', 'услуга', 'заявка', 'знать', 'просто', 'обслуживание', 'очередь', 'минута', 'обратиться', 'данный', 'большой', 'первый', 'сегодня', 'девушка', 'ждать', 'сайт', 'руб', 'претензия', 'новый', 'просить', 'интернет', 'процент', 'неделя', 'никакой', 'оператор', 'сообщить', 'ответить', 'являться', 'далее', 'стать', 'нужно', 'час', 'несколько', 'оформить', 'перевод', 'писать', 'комиссия', 'следу

In [23]:
top_words_vec = model_response.wv[top_words]
tsne = TSNE(n_components=2, random_state=21)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [24]:
output_notebook()

p = figure(tools = 'pan, wheel_zoom, reset, save',
           toolbar_location = 'above',
           title = 'TSNE для топ-500 слов')

source = ColumnDataSource(data = dict(x=top_words_tsne[:,0],
                                      y=top_words_tsne[:,1],
                                      names=top_words))

p.scatter(x='x', y='y', size=6, source=source)

labels = LabelSet(x='x', y='y', text = 'names', y_offset = 6,
                  text_font_size = '8pt', text_color = 'blue',
                  source = source, text_align = 'center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

**Примеры осмысленных кластеров**

![Временные кластеры](./plots/Data_TSNE-plot.png)
![Денежный кластер](./plots/sum_TSNE-plot.png)

**Вывод**: легко выделяются кластера слов с месяцами года, относящиеся к дате, относящиеся к денежной мере и относящиеся к штрафам.

- задать координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.

In [25]:
bank_list = ['сбербанк','альфа','втб','бинбанк', 'хоум', 'тинькофф',
             'ренессанс', 'отп', 'ситибанк', 'авангард',
             'промсвязьбанк', 'юникредит', 'мтс', 'росбанк', 'уралсиб']
bank_list_vec = model_response.wv[bank_list]

In [26]:
x_axis = model_response.wv['хороший'] - model_response.wv['плохой']
y_axis =  model_response.wv['быстрый'] - model_response.wv['медленный']

bank_list_x = []
bank_list_y = []
for el in bank_list_vec:
    bank_list_x.append(np.dot(el, x_axis))
    bank_list_y.append(np.dot(el, y_axis))
    
p = figure(tools="pan, wheel_zoom, reset, save",
           toolbar_location="above",
           x_axis_label = "плохой --> хороший",
           y_axis_label = "медленный --> быстрый",
           title="TSNE для банков")

source = ColumnDataSource(data=dict(x=bank_list_x,
                                    y=bank_list_y,
                                    names=bank_list))

p.scatter(x="x", y="y", size=8, source=source)

labels = LabelSet(x="x", y="y", text="names", y_offset=6,
                  text_font_size="8pt", text_color="blue",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

![Банки](./plots/Banks_TSNE-plot_new.png)

### Часть 2: Распространение метки.
#### Создание оценочного словаря

Будем считать, что положительному классу соответствует 1, а негативному соответствует 0

In [ ]:
positive = ['быстрый', 'хороший', 'легко', 'комфортно',
            'отлично', 'нравиться','удобный', 'отличный']
negative = ['долго', 'медленный', 'плохой', 'ужасно',
            'отвратительный', 'тяжело', 'скверный', 'безобразно']

#### Построение лексического графа.

In [56]:
g = ig.Graph(directed=True)
for word in model_response.wv.key_to_index.keys():
    g.add_vertex(word)

for word in tqdm(model_response.wv.key_to_index.keys()):
    node = g.vs.select(name = word).indices[0]
    similar_words = model_response.wv.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)

100%|██████████████████████████████████████████████████████████████████████████| 42565/42565 [1:28:32<00:00,  8.01it/s]


In [57]:
ig.summary(g)

IGRAPH DNW- 42565 212825 -- 
+ attr: name (v), weight (e)


Обход ограничения на размер вывода при использовании графа.

In [58]:
with open('out.txt', 'w') as f:
    with redirect_stdout(f):
        print(g)

In [59]:
with open('out.txt') as f:
  l = [line.strip() for line in f]

In [60]:
l[:20]

['IGRAPH DNW- 42565 212825 --',
 '+ attr: name (v), weight (e)',
 '+ edges (vertex names):',
 'банк -> сбербанк, ситибанк, росбанк, бинбанка, брс',
 'карта -> карточка, дебетовый, кредитка, кк, дебетовка',
 'это -> данный, хотя, вообще, подобный, невосполнять',
 'деньга -> средство, денежка, наличка, дс, денюжка',
 'день -> месяц, неделя, час, сутки, четверг',
 'всё -> вс, ладный, вроде, нифига, воспрепятствование',
 'кредит -> ипотека, автокредит, рассрочка, потребкредить,',
 'ссуда',
 'который -> хотя, поскольку, соответственно, следовательно,',
 'посторонний',
 'отделение -> офис, филиал, осб, допофис, отд',
 'клиент -> вкладчик, пользователь, мщиковы, клиентура,',
 'клиенто',
 'сотрудник -> специалист, менеджер, сотрудница, работник,',
 'работница',
 'мочь -> хотеть, нужно, смочь, возможно, невозможно',
 'счёт -> сч, сберкнижка, картсчёт, скс, картсча']

#### Пометить выбранные слова в лексическом графе соответствующими метками.

In [154]:
def marking():
    '''Маркировка'''
    for word in tqdm(words):
        label = int(spread_df['label'][spread_df['word'] == word])
        if label != -1:
            get_label([word], label)

def get_label(word, label):
    '''
    Ищит близкие слова. Если близость больше 0.6, то выставляет матку, как у первоначального слова.
    '''
    seq_s = model_response.wv.most_similar(word, topn=10)
    for seq in seq_s:
        if seq[1] > 0.6:
            spread_df['label'][spread_df['word'] == seq[0]] = label

Первый этап маркировки словаря на основу отобранных слов.

In [190]:
words = model_response.wv.index_to_key
vectors = [model_response.wv[i] for i in words]
labels = []
for w in words:
  if w in positive: labels.append(1)
  elif w in negative: labels.append(0)
  else: labels.append(-1)

In [258]:
spread_df = pd.DataFrame(zip(words, labels), columns=['word', 'label'])

Второй этап маркировки.

In [259]:
marking()

100%|████████████████████████████████████████████████████████████████████████████| 42565/42565 [07:44<00:00, 91.67it/s]


In [260]:
spread_df[spread_df['label'] != -1]

,word,label
36,очень,0
127,хороший,1
310,долго,0
459,удобный,1
524,довольный,1
...,...,...
42436,руднева,0
42500,поубивать,0
42527,тритий,1
42551,совокбанк,1


Третий этап маркировки.

In [261]:
marking()

100%|████████████████████████████████████████████████████████████████████████████| 42565/42565 [17:20<00:00, 40.89it/s]


In [262]:
spread_df[spread_df['label'] != -1]

,word,label
36,очень,0
127,хороший,1
310,долго,0
319,маленький,1
342,вежливый,1
...,...,...
42539,годок,1
42540,жирнов,0
42547,ностальдж,1
42551,совокбанк,1


Число промаркироваанных слов у нас получилась достаточной для LabelSpreading, поэтому на четвёртом этапе маркировки используем его.

In [266]:
label_model = LabelSpreading(kernel='knn', alpha=0.2, n_neighbors=7, n_jobs=4, max_iter=400)

In [277]:
model_df = pd.concat([spread_df, pd.DataFrame(vectors)], axis=1)

In [279]:
model_df = model_df[model_df['label'] != -1]
y = model_df['label']
X = model_df.drop(columns=['word', 'label'])

In [280]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [281]:
label_model.fit(X_train, y_train)

LabelSpreading(kernel='knn', max_iter=400, n_jobs=4)

Произведём маркировку лексического словаря на основе обученной модели LabelSpreading и проверим её точность.

In [282]:
spread_df['prediction'] = spread_df.word.apply(lambda x: label_model.predict([model_response.wv[x]])[0])

In [283]:
predict = label_model.predict(X_test)
report = classification_report(y_test, predict)
print(report)

              precision    recall  f1-score   support

           0       0.81      0.66      0.73       214
           1       0.86      0.93      0.89       467

    accuracy                           0.85       681
   macro avg       0.83      0.80      0.81       681
weighted avg       0.84      0.85      0.84       681



In [286]:
result = spread_df[spread_df['label'] == -1]
result.head(20)

,word,prediction
0,банк,1
1,карта,1
2,это,1
3,деньга,1
4,день,1
5,всё,1
6,кредит,1
7,который,0
8,отделение,1
9,клиент,1


### Выводы:
В рамках работы был составлен словарь для последующей классификации по тональности. На основе словаря были построены визуализации с применением TSNE.

Также была произведено распространение метки в лексическом словаре: до применение модели LabelPropagation была произведена разметка словаря по ближайшим словам к размеченным. Точность модели была получена равная 0.85